In [1]:
from grpo import *

/u/jgottesman/.conda/envs/grposloth_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


INFO 04-01 22:51:16 [__init__.py:239] Automatically detected platform cuda.


2025-04-01 22:51:16,824	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [2]:
dataset = get_TSP_questions("tsp_training_prompt_dataset.json")
test_dataset = get_TSP_questions("tsp_test_prompt_dataset.json")


#model_name = "meta-llama/Llama-3.2-1B-Instruct"
model_name = "unsloth/Qwen2.5-3B-Instruct-bnb-4bit"


if torch.cuda.is_available():
    device = "cuda"
elif torch.backends.mps.is_available():
    device = "mps"
else:
    device = "cpu"

if "Llama" in model_name:
    output_dir = "outputs/Llama-1B-GRPO"
    run_name = "Llama-1B-GRPO-TSP"
else:
    output_dir="outputs/Qwen-3B-GRPO"
    run_name="Qwen-3B-GRPO-TSP"
    
training_args = GRPOConfig(
    output_dir=output_dir,
    run_name=run_name,
    learning_rate=5e-6,
    adam_beta1 = 0.9,
    adam_beta2 = 0.99,
    weight_decay = 0.1,
    warmup_ratio = 0.1,
    lr_scheduler_type='cosine',
    logging_steps=1,
    bf16=True,
    per_device_train_batch_size=8,
    gradient_accumulation_steps=2,
    num_generations=4,
    max_prompt_length=600,
    max_completion_length=1022,
    num_train_epochs=1,
    save_steps=100,
    max_grad_norm=0.1,
    report_to="wandb",
    log_on_each_node=False,
    beta = 0
)

peft_config = LoraConfig(
    r=16,
    lora_alpha=64,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "up_proj", "down_proj", "gate_proj"],
    task_type="CAUSAL_LM",
    lora_dropout=0.05,
)

from unsloth import FastLanguageModel
import torch
max_seq_length = 2500 
lora_rank = 16 

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_name,
    max_seq_length = max_seq_length,
    load_in_4bit = True, # False for LoRA 16bit
    fast_inference = True, # Enable vLLM fast inference
    max_lora_rank = lora_rank,
    gpu_memory_utilization = 0.5, # Reduce if out of memory
)

model = FastLanguageModel.get_peft_model(
    model,
    r = lora_rank, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = [
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ], # Remove QKVO if out of memory
    lora_alpha = lora_rank,
    use_gradient_checkpointing = "unsloth", # Enable long context finetuning
    random_state = 3407,
)

# use peft at your own risk; not working for me with multi-GPU training
trainer = GRPOTrainer(
    model=model,
    processing_class=tokenizer,
    reward_funcs=[
        valid_response_reward_func,
        soft_format_reward_func,
        strict_format_reward_func,
        optimal_solution_reward,
        improvement_reward_func],
    args=training_args,
    train_dataset=dataset,
    eval_dataset = test_dataset
)
trainer.train()

/tmp/ipykernel_1602808/830294111.py:55: UserWarning: WARNING: Unsloth should be imported before trl, transformers, peft to ensure all optimizations are applied. Your code may run slower or encounter memory issues without these optimizations.

Please restructure your imports with 'import unsloth' at the top of your file.
  from unsloth import FastLanguageModel


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.3.19: Fast Qwen2 patching. Transformers: 4.50.3. vLLM: 0.8.2.
   \\   /|    NVIDIA A100-SXM4-40GB. Num GPUs = 1. Max memory: 39.381 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.0. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post2. FA2 = True]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: vLLM loading unsloth/qwen2.5-3b-instruct-bnb-4bit with actual GPU utilization = 49.42%
Unsloth: Your GPU has CUDA compute capability 8.0 with VRAM = 39.38 GB.
Unsloth: Using conservativeness = 1.0. Chunked prefill tokens = 2500. Num Sequences = 288.
Unsloth: vLLM's KV Cache can use up to 17.21 GB. Also swap space = 6 GB.
INFO 04-01 22:51:41 [config.py:585] This model su

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  2.20it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  2.19it/s]

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  1.87it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  1.86it/s]


INFO 04-01 22:51:44 [punica_selector.py:18] Using PunicaWrapperGPU.


INFO 04-01 22:51:44 [model_runner.py:1146] Model loading took 2.0011 GB and 2.634143 seconds
INFO 04-01 22:51:46 [worker.py:267] Memory profiling takes 1.24 seconds
INFO 04-01 22:51:46 [worker.py:267] the current vLLM instance can use total_gpu_memory (39.38GiB) x gpu_memory_utilization (0.49) = 19.46GiB
INFO 04-01 22:51:46 [worker.py:267] model weights take 2.00GiB; non_torch_memory takes 0.09GiB; PyTorch activation peak memory takes 1.57GiB; the rest of the memory reserved for KV Cache is 15.80GiB.
INFO 04-01 22:51:46 [executor_base.py:111] # cuda blocks: 28771, # CPU blocks: 10922
INFO 04-01 22:51:46 [executor_base.py:116] Maximum concurrency for 2500 tokens per request: 184.13x
INFO 04-01 22:51:51 [model_runner.py:1442] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI. If out-of-memory error occurs during cudagraph capture, consider dec

...


Unsloth 2025.3.19 patched 36 layers with 36 QKV layers, 36 O layers and 36 MLP layers.
Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 990 | Num Epochs = 1 | Total steps = 247
O^O/ \_/ \    Batch size per device = 8 | Gradient accumulation steps = 2
\        /    Data Parallel GPUs = 1 | Total batch size (8 x 2 x 1) = 16
 "-____-"     Trainable parameters = 29,933,568/3,000,000,000 (1.00% trained)
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: jacobgott6 (jacobgott6-northeastern-university) to https://api.wandb.ai. 

`generation_config` default values have been modified to match model-specific defaults: {'max_length': 32768, 'eos_token_id': 151645}. If this is not desired, please set these values explicitly.


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,0.000000
2,-0.000000
3,0.000000
4,0.000000
5,-0.000000
6,0.000000
7,0.000000
8,-0.000000
9,-0.000000
10,0.000000


TrainOutput(global_step=247, training_loss=-7.013198335161093e-10, metrics={'train_runtime': 23335.2765, 'train_samples_per_second': 0.042, 'train_steps_per_second': 0.011, 'total_flos': 0.0, 'train_loss': -7.013198335161093e-10})